In [ ]:
#Author: Tongshu Zheng from Duke University
#Email: tongshu.zheng@duke.edu; contact me if you have any questions regarding the code
#Please reference the code source and publication (i.e., "Estimating ground-level PM2.5 using micro-satellite 
#images by a convolutional neural network and random forest approach") if you use the code.

In [ ]:
# To support both python 2 and python 3
from __future__ import division, print_function, unicode_literals

# Common imports
import numpy as np
import os
# to make this notebook's output stable across runs
np.random.seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12
plt.rcParams['font.family'] = 'sans-serif'
plt.rcParams['font.sans-serif'] = 'Arial'
# Where to save the figures
PROJECT_ROOT_DIR = "."
PROJECT_SAVE_DIR = "Project"

import os
if not (os.path.isdir(PROJECT_ROOT_DIR+'/'+PROJECT_SAVE_DIR)):
    print('Figure directory didn''t exist, creating now.')
    os.mkdir(PROJECT_ROOT_DIR+'/'+PROJECT_SAVE_DIR)
else:
    print('Figure directory exists.') 
    
def savepdf(fig,name):
    fig.savefig(PROJECT_ROOT_DIR+'/'+PROJECT_SAVE_DIR+'/'+name+'.pdf')

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

from itertools import compress
from scipy import stats
import copy
import cv2
import math
import pandas as pd
from sklearn.neighbors import LocalOutlierFactor
from scipy.interpolate import interp1d
from datetime import datetime
from datetime import timedelta
from sklearn.linear_model import LinearRegression
from sklearn import preprocessing
import os
import requests
from requests.auth import HTTPBasicAuth
import json
from multiprocessing.dummy import Pool as ThreadPool
import time
import shlex, subprocess
import urllib.request
import webbrowser
import copy
import imageio
from os import listdir
from os.path import isfile, join
import tensorflow as tf
import random
import keras
from keras import applications
from keras.utils import multi_gpu_model
from keras.optimizers import RMSprop, SGD
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import  image
from keras import optimizers
from keras.models import Sequential, Model, load_model
from keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D, GlobalMaxPooling2D, Conv2D, MaxPooling2D, Activation
from keras import backend as k 
from keras.callbacks import ModelCheckpoint, Callback,LearningRateScheduler, TensorBoard, EarlyStopping
from keras.layers.normalization import BatchNormalization
from keras.applications.vgg16 import VGG16
from keras.applications.vgg19 import VGG19
from keras.applications.vgg19 import preprocess_input


# Load in the Images 

In [ ]:
my_X_images_folder = './X_image_full'
image_files = [f for f in listdir(my_X_images_folder) if isfile(join(my_X_images_folder, f)) and '.DS_Store' not in f ]
image_files_sorted = sorted(image_files, key = lambda x: int(x.split('.')[0].split('_')[2]))

image_files_sorted

# Use keras to preprocess images so that they have the correct forms for the ImageNet CNN models later

In [ ]:
#image_size = 67 #200 m * 200 m model
#image_size = 167 #500 m * 500 m model
image_size = 224 #670 m * 670 m model
#image_size = 33 #100 m * 100 m model
X_image = []
for image_file in image_files_sorted:
    img = image.load_img(my_X_images_folder+'/'+image_file,target_size=(image_size, image_size))
    x = image.img_to_array(img) #(224,224,3)
    x = np.expand_dims(x, axis=0) #(1,224,224,3)
    x = preprocess_input(x)
    X_image.append(x)

In [ ]:
y_PM25 = []
with open("./y_PM25_full.txt", "r") as f:
    for line in f:
        y_PM25.append(np.float32(float(line.strip())))

In [ ]:
## Train-Test Split

dataset = list(zip(X_image, y_PM25))

###shuffle dataset to get train, test datasets
random.Random(42).shuffle(dataset)
#random.shuffle(dataset)
batch_size = 100
total_batches = len(dataset) // batch_size
train_batches = int(np.ceil(8 * total_batches / 10))
test_batches = total_batches - train_batches
train, test = dataset[:train_batches*batch_size], dataset[train_batches*batch_size:]

In [ ]:
X_train = np.array([item[0] for item in train]).reshape(-1,image_size,image_size,3)
X_test = np.array([item[0] for item in test]).reshape(-1,image_size,image_size,3)
y_train = np.array([item[1] for item in train]).reshape(-1)
y_test = np.array([item[1] for item in test]).reshape(-1)

# Iterative fine-tuning VGG16 for an exxample 670 m resolution model (the 500 m and 200 m models are pretty much the same)

In [ ]:
VGG_finetune = VGG16(include_top=False, weights='imagenet', \
                            input_tensor=None, input_shape=(image_size, image_size, 3), pooling='max', classes=1000)

In [ ]:
for layer in VGG_finetune.layers[:]:
    layer.trainable = False

In [ ]:
x = VGG_finetune.output
x = Dropout(0.5)(x) # 500, 200m models = 0.1
x = Dense(128, activation="relu")(x) #500=100, 200 m =80 
x = Dropout(0.5)(x)
predictions = Dense(1, activation=None)(x)
# creating the final model 
model_final = Model(input = VGG_finetune.input, output = predictions)
model_final.compile(optimizer=optimizers.Adam(), loss='mean_squared_error', metrics=['mse'])

model_final.summary()

In [ ]:
train_datagen = ImageDataGenerator(
                rotation_range=30, width_shift_range=0.1, height_shift_range=0.1, zoom_range = 0.5,\
                            fill_mode = 'reflect', horizontal_flip = True,vertical_flip=True, channel_shift_range = 0)

test_datagen = ImageDataGenerator()

In [ ]:
train_generator = train_datagen.flow(X_train, y_train, batch_size=batch_size)
validation_generator = test_datagen.flow(X_test, y_test, batch_size=batch_size)

In [ ]:
checkpoint = ModelCheckpoint(filepath='./checkpoint_VGG16_finetune_670m/checkpoint-{epoch:02d}-{val_loss:.2f}.hdf5', monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)
#early = EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=1, mode='auto')
class LossHistory(Callback):
    def on_train_begin(self, logs={}):
        self.losses = []

    def on_batch_end(self, batch, logs={}):
        self.losses.append(logs.get('loss'))
history = LossHistory()

In [ ]:
model_final.fit_generator(train_generator, steps_per_epoch=len(X_train)//batch_size, epochs=60, verbose=1, callbacks=[checkpoint,history], validation_data=validation_generator, validation_steps=len(y_train)//batch_size
              )

# What about retrain the last conv. layer

In [ ]:
VGG_finetune = VGG16(include_top=False, weights='imagenet', \
                            input_tensor=None, input_shape=(image_size, image_size, 3), pooling='max', classes=1000)

In [ ]:
for layer in VGG_finetune.layers[:-3]:
    layer.trainable = False
for layer in VGG_finetune.layers[-3:]:
    layer.trainable = True

In [ ]:
VGG_finetune.summary()

In [ ]:
# load your best model from the last training here
model = load_model('') #670m

In [ ]:
weights_128 = model.layers[-3].get_weights()

weights_1 = model.layers[-1].get_weights()

In [ ]:
model.summary()

# then

In [ ]:
x = VGG_finetune.output
x = Dropout(0.5)(x)
x = Dense(128, activation="relu",weights = weights_128)(x) 
x = Dropout(0.5)(x)
predictions = Dense(1, activation=None, weights = weights_1)(x)
# creating the final model 
model_final = Model(input = VGG_finetune.input, output = predictions)
model_final.compile(optimizer=optimizers.Adam(), loss='mean_squared_error', metrics=['mse'])

model_final.summary()

In [ ]:
for layer in model_final.layers[-3:]:
    layer.trainable = False

In [ ]:
model_final.compile(optimizer=optimizers.Adam(), loss='mean_squared_error', metrics=['mse'])

model_final.summary()

In [ ]:
train_datagen = ImageDataGenerator(
                rotation_range=30, width_shift_range=0.1, height_shift_range=0.1, zoom_range = 0.5,\
                            fill_mode = 'reflect', horizontal_flip = True,vertical_flip=True, channel_shift_range = 0)

test_datagen = ImageDataGenerator()

In [ ]:
train_generator = train_datagen.flow(X_train, y_train, batch_size=batch_size)
validation_generator = test_datagen.flow(X_test, y_test, batch_size=batch_size)

In [ ]:
checkpoint = ModelCheckpoint(filepath='./checkpoint_VGG16_finetune_670m_last_conv/checkpoint-{epoch:02d}-{val_loss:.2f}.hdf5', monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)
#early = EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=1, mode='auto')
class LossHistory(Callback):
    def on_train_begin(self, logs={}):
        self.losses = []

    def on_batch_end(self, batch, logs={}):
        self.losses.append(logs.get('loss'))
history = LossHistory()

In [ ]:
model_final.fit_generator(train_generator, steps_per_epoch=len(X_train)//batch_size, epochs=60, verbose=1, callbacks=[checkpoint,history], validation_data=validation_generator, validation_steps=len(y_train)//batch_size
              )

# What about further retraining the second to last conv. layer?

In [ ]:
# again load your best model from the last training here
model = load_model('') #670m

In [ ]:
model.summary()

In [ ]:
weights_lastconv = model.layers[-7].get_weights()

In [ ]:
VGG_finetune = VGG16(include_top=False, weights='imagenet', \
                            input_tensor=None, input_shape=(image_size, image_size, 3), pooling='max', classes=1000)

In [ ]:
for layer in VGG_finetune.layers[:-4]:
    layer.trainable = False
for layer in VGG_finetune.layers[-3:]:
    layer.trainable = False

In [ ]:
VGG_finetune.layers[-3].set_weights(weights_lastconv)

In [ ]:
VGG_finetune.summary()

In [ ]:
weights_128 = model.layers[-3].get_weights()
weights_1 = model.layers[-1].get_weights()

# then

In [ ]:
x = VGG_finetune.output
x = Dropout(0.5)(x)
x = Dense(128, activation="relu",  weights = weights_128)(x) #500m = 100
x = Dropout(0.5)(x)
predictions = Dense(1, activation=None, weights = weights_1)(x)
# creating the final model 
model_final = Model(input = VGG_finetune.input, output = predictions)
model_final.compile(optimizer=optimizers.Adam(), loss='mean_squared_error', metrics=['mse'])

model_final.summary()

In [ ]:
for layer in model_final.layers[-3:]:
    layer.trainable = False

In [ ]:
model_final.compile(optimizer=optimizers.Adam(), loss='mean_squared_error', metrics=['mse'])

model_final.summary()

In [ ]:
train_datagen = ImageDataGenerator(
                rotation_range=30, width_shift_range=0.1, height_shift_range=0.1, zoom_range = 0.5,\
                            fill_mode = 'reflect', horizontal_flip = True,vertical_flip=True, channel_shift_range = 0)

test_datagen = ImageDataGenerator()

In [ ]:
train_generator = train_datagen.flow(X_train, y_train, batch_size=batch_size)
validation_generator = test_datagen.flow(X_test, y_test, batch_size=batch_size)

In [ ]:
checkpoint = ModelCheckpoint(filepath='./checkpoint_VGG16_finetune_670m_seclast_conv/checkpoint-{epoch:02d}-{val_loss:.2f}.hdf5', monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)
#early = EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=1, mode='auto')
class LossHistory(Callback):
    def on_train_begin(self, logs={}):
        self.losses = []

    def on_batch_end(self, batch, logs={}):
        self.losses.append(logs.get('loss'))
history = LossHistory()

In [ ]:
model_final.fit_generator(train_generator, steps_per_epoch=len(X_train)//batch_size, epochs=60, verbose=1, callbacks=[checkpoint,history], validation_data=validation_generator, validation_steps=len(y_train)//batch_size
              )

# What about further retraining the third to last conv. layer?


In [ ]:
# again load your best model from the last training here
model = load_model('') #670m

In [ ]:
model.summary()

In [ ]:
weights_lastconv = model.layers[-7].get_weights()

In [ ]:
weights_2tolastconv = model.layers[-8].get_weights()

In [ ]:
weights_128 = model.layers[-3].get_weights()
weights_1 = model.layers[-1].get_weights()

In [ ]:
VGG_finetune = VGG16(include_top=False, weights='imagenet', \
                            input_tensor=None, input_shape=(image_size, image_size, 3), pooling='max', classes=1000)

In [ ]:
for layer in VGG_finetune.layers[:-5]:
    layer.trainable = False
for layer in VGG_finetune.layers[-4:]:
    layer.trainable = False

In [ ]:
VGG_finetune.layers[-3].set_weights(weights_lastconv)
VGG_finetune.layers[-4].set_weights(weights_2tolastconv)

In [ ]:
VGG_finetune.summary()

# then

In [ ]:
x = VGG_finetune.output
x = Dropout(0.5)(x)
x = Dense(128, activation="relu", weights = weights_128)(x)
x = Dropout(0.5)(x)
predictions = Dense(1, activation=None, weights = weights_1)(x)
# creating the final model 
model_final = Model(input = VGG_finetune.input, output = predictions)

In [ ]:
for layer in model_final.layers[-3:]:
    layer.trainable = False

In [ ]:
model_final.compile(optimizer=optimizers.Adam(), loss='mean_squared_error', metrics=['mse'])


In [ ]:
model_final.summary()

In [ ]:
train_datagen = ImageDataGenerator(
                rotation_range=30, width_shift_range=0.1, height_shift_range=0.1, zoom_range = 0.5,\
                            fill_mode = 'reflect', horizontal_flip = True,vertical_flip=True, channel_shift_range = 0)
test_datagen = ImageDataGenerator()

In [ ]:
train_generator = train_datagen.flow(X_train, y_train, batch_size=batch_size)
validation_generator = test_datagen.flow(X_test, y_test, batch_size=batch_size)

In [ ]:
checkpoint = ModelCheckpoint(filepath='./checkpoint_VGG16_finetune_670m_thirdlast_conv/checkpoint-{epoch:02d}-{val_loss:.2f}.hdf5', monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)
#early = EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=1, mode='auto')
class LossHistory(Callback):
    def on_train_begin(self, logs={}):
        self.losses = []

    def on_batch_end(self, batch, logs={}):
        self.losses.append(logs.get('loss'))
history = LossHistory()

In [ ]:
model_final.fit_generator(train_generator, steps_per_epoch=len(X_train)//batch_size, epochs=80, verbose=1, callbacks=[checkpoint,history], validation_data=validation_generator, validation_steps=len(y_train)//batch_size
              )

# RF part and final Model Evaluation

In [ ]:
# load the final best model here!

#670m
model = load_model('')


In [ ]:
model.summary()
extract_model = Model(inputs=model.input, outputs=model.get_layer('dense_1').output) #may not be dense_1, 
# should be the dense layer with dimension 128 for 670 m model

In [ ]:
X_image_features = extract_model.predict(np.array(X_image).reshape(-1,image_size,image_size,3))

In [ ]:
len(X_image_features)

In [ ]:
np.shape(X_image_features)

In [ ]:
X_image_features



# What if I use meteorology features?

In [ ]:
meteo_feature = np.load('meteo_feature_full.npy')

In [ ]:
len(meteo_feature)

In [ ]:
features_meteo = np.append(X_image_features,meteo_feature,axis = 1)

In [ ]:
mask = ~np.any(np.isnan(features_meteo), axis=1)
features_meteo = features_meteo[mask]

In [ ]:
y_all= np.array(y_PM25).reshape([13022,-1])[mask]

In [ ]:
np.shape(features_meteo)

In [ ]:
y_all

## Random Forest Regression

In [ ]:
from sklearn.metrics import mean_squared_error
from sklearn.feature_selection import SelectKBest
from sklearn.pipeline import Pipeline
from sklearn.model_selection import StratifiedKFold, KFold, cross_val_score
from sklearn import metrics
import pandas as pd
import scipy
from sklearn.neighbors import LocalOutlierFactor
from datetime import datetime
from datetime import timedelta
from sklearn.linear_model import LinearRegression
from sklearn import preprocessing
from scipy import stats
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split


from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import make_scorer
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import median_absolute_error
from sklearn.preprocessing import MinMaxScaler

In [ ]:
## load in site labels and time index to see the seasonal trend and site-specific trend
time_stamps = np.load('time_stamp_full.npy')
site_labels = np.load('site_label.npy')

In [ ]:
time_stamps = np.array([float(x) for x in time_stamps ])
site_labels = np.array([float(x) for x in site_labels ])

In [ ]:
features_meteo = np.append(features_meteo,time_stamps.reshape(13022,1),axis = 1)
features_meteo = np.append(features_meteo,site_labels.reshape(13022,1),axis = 1)

In [ ]:
features_meteo

# Try to match RF's train test splits with the CNN's splits

In [ ]:
dataset_RF = list(zip(features_meteo, y_all))

###shuffle dataset to get train, test datasets
random.Random(42).shuffle(dataset_RF)


batch_size = 100
total_batches = len(dataset_RF) // batch_size
train_batches = int(np.ceil(8 * total_batches / 10))
test_batches = total_batches - train_batches

In [ ]:
train_full, test_full = dataset_RF[:train_batches*batch_size], dataset_RF[train_batches*batch_size:]
X_train_full = np.array([x[0] for x in train_full])
X_test_full = np.array([x[0] for x in test_full])
y_train = np.array([x[1] for x in train_full])
y_test = np.array([x[1] for x in test_full])
X_train = X_train_full[:, :-2]
X_test = X_test_full[:, :-2]

In [ ]:
np.mean(y_test)

In [ ]:
shuffle=KFold(n_splits=5, shuffle=True,random_state=42)
max_features=['auto']
n_estimators=['300','400','500']
#min_samples_leaf=np.linspace(2,8,3)
min_samples_leaf=['1','2','3']
avg_MSE=np.zeros([len(max_features),len(n_estimators),len(min_samples_leaf)])
avg_MAE = np.zeros([len(max_features),len(n_estimators),len(min_samples_leaf)])
for k, feature in enumerate(max_features):
    for i,estimators in enumerate(n_estimators):
        for j,samples_leaf in enumerate(min_samples_leaf):
            print('k = '+str(k)+', i = '+str(i)+', j = '+str(j)+' started.')
            scaler=StandardScaler()
            cls = RandomForestRegressor(bootstrap=True, n_jobs=-1, random_state = 40,max_features =feature,n_estimators =int(estimators),min_samples_leaf=int(samples_leaf ) )
            our_pipeline=Pipeline([('scaler',scaler),('cls',cls)])
            MSE = -cross_val_score(our_pipeline,X_train, y_train.ravel(), cv=shuffle, scoring='neg_mean_squared_error')
            avg_MSE[k,i,j]=np.mean(MSE)
            #MAE = -cross_val_score(our_pipeline,X_train, y_train.ravel(), cv=shuffle, scoring='neg_mean_absolute_error')
            #avg_MAE[k,i,j]=np.mean(MAE)
            print('k = '+str(k)+', i = '+str(i)+', j = '+str(j)+' finished.')

In [ ]:
np.sqrt(avg_MSE)

# Compute 5-fold results

In [ ]:
def final_cv_results(X,y):
    CV5_spearman = []
    CV5_pearson = []
    CV5_MAE = []
    CV5_NMAE = []
    CV5_RMSE = []
    CV5_NRMSE = []
    for train_index, test_index in cv.split(X, y):
        print("TRAIN:", train_index, "TEST:", test_index)
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        scaler=StandardScaler()
        cls = RandomForestRegressor(bootstrap=True, n_jobs=-1, random_state = 40,max_features ='auto',n_estimators =500,min_samples_leaf=1 )
        our_pipeline=Pipeline([('scaler',scaler),('cls',cls)])
        our_pipeline.fit(X_train,y_train.ravel())
        CV5_spearman.append(stats.spearmanr((our_pipeline.predict(X_test)),y_test.ravel())[0])
        CV5_pearson.append(stats.pearsonr((our_pipeline.predict(X_test)),y_test.ravel())[0])
        my_prediction = our_pipeline.predict(X_test)
        RMSE = round(np.sqrt(metrics.mean_squared_error(y_test, my_prediction)),1)
        MAE = round(metrics.mean_absolute_error(y_test, my_prediction),1)
        CV5_MAE.append(MAE)
        CV5_NMAE.append(MAE/np.mean(y_test))
        CV5_RMSE.append(RMSE)
        CV5_NRMSE.append(RMSE/np.mean(y_test))

    return  CV5_spearman, CV5_pearson, CV5_MAE, CV5_NMAE, CV5_RMSE, CV5_NRMSE

In [ ]:
 CV5_spearman, CV5_pearson, CV5_MAE, CV5_NMAE, CV5_RMSE, CV5_NRMSE = final_cv_results(X_train,y_train)

In [ ]:
CV5_mean = [np.mean(CV5_spearman),np.mean(CV5_pearson),np.mean(CV5_MAE), np.mean(CV5_NMAE), np.mean(CV5_RMSE), np.mean(CV5_NRMSE) ]

In [ ]:
CV5_mean

# Illustrating 5-fold CV results

In [ ]:
from matplotlib.patches import Patch
cmap_data = plt.cm.Paired
cmap_cv = plt.cm.coolwarm

In [ ]:
def plot_cv_indices(cv, X, y, ax, n_splits, lw=40):
    """Create a sample plot for indices of a cross-validation object."""

    # Generate the training/testing visualizations for each CV split
    for ii, (tr, tt) in enumerate(cv.split(X=X, y=y)):
        # Fill in indices with the training/test groups
        indices = np.array([np.nan] * len(X))
        indices[tt] = 1
        indices[tr] = 0

        # Visualize the results
        ax.scatter(range(len(indices)), [ii + .5] * len(indices),
                   c=indices, marker='_', lw=lw, cmap=cmap_cv,
                   vmin=-.2, vmax=1.2)
    for ii, value in enumerate(['Fold 0', 'Fold 1', 'Fold 2', 'Fold 3', 'Fold 4']):
        plt.text(1000+2000*ii, 6-0.4,value,horizontalalignment='center', fontsize=20, rotation=0, color ='black' )
    for ii, value in enumerate(CV5_spearman):
        plt.text(1000+2000*ii, 7-0.4,round(value,2),horizontalalignment='center', fontsize=20, rotation=0, color ='black' )
    for ii, value in enumerate(CV5_pearson):
        plt.text(1000+2000*ii, 8-0.4,round(value,2),horizontalalignment='center', fontsize=20, rotation=0, color ='black' )
    for ii, value in enumerate(CV5_MAE):
        plt.text(1000+2000*ii, 9-0.4,round(value,1),horizontalalignment='center', fontsize=20, rotation=0, color ='black' )
    for ii, value in enumerate(CV5_NMAE):
        plt.text(1000+2000*ii, 10-0.4,str(round(value*100,1))+'%',horizontalalignment='center', fontsize=20, rotation=0, color ='black' )
    for ii, value in enumerate(CV5_RMSE):
        plt.text(1000+2000*ii, 11-0.4,round(value,1),horizontalalignment='center', fontsize=20, rotation=0, color ='black' )
    for ii, value in enumerate(CV5_NRMSE):
        plt.text(1000+2000*ii, 12-0.4,str(round(value*100,1))+'%',horizontalalignment='center', fontsize=20, rotation=0, color ='black' )
    ''' 
    for ii, value in enumerate(CV5_mean):
        if ii==0 or 1:
            plt.text(1000+2000*ii, 12-0.2,round(value,2),horizontalalignment='center', fontsize=15, rotation=0, color ='black' )
        elif ii==2 or 4:
            plt.text(1000+2000*ii, 12-0.2,round(value,1),horizontalalignment='center', fontsize=15, rotation=0, color ='black' )

        else:
            plt.text(1000+2000*ii, 12-0.2,str(round(value*100,1))+'%',horizontalalignment='center', fontsize=15, rotation=0, color ='black' )
    '''
    # Plot the data classes and groups at the end
    #ax.scatter(range(len(X)), [ii + 1.5] * len(X),
    #           c=y, marker='_', lw=lw, cmap=cmap_data)

    #ax.scatter(range(len(X)), [ii + 2.5] * len(X),
    #           c=group, marker='_', lw=lw, cmap=cmap_data)

    # Formatting
    yticklabels = list(range(n_splits)) + ['Folds','Spearman r','Pearson r','MAE', 'NMAE','RMSE','NRMSE']
    ax.set(yticks=np.arange(n_splits+7) + .5, yticklabels=yticklabels,
           xlabel='Sample index', ylabel="CV iteration",
           ylim=[n_splits+7.2, -.2], xlim=[0, len(X)])
    ax.legend([Patch(color=cmap_cv(.8)), Patch(color=cmap_cv(.02))],
              ['CV test', 'CV training'], loc=(-0.5, 0.8), fontsize=20)
    ax.set_title('5-fold CV results on the Beijing training set', fontsize=20)
    ax.tick_params(labelsize = 20)
    ax.set_xlabel('Sample index',size = 20)
    ax.set_ylabel("CV iteration", size = 20)
    plt.tight_layout()
    return ax

In [ ]:
n_splits = 5
fig, ax = plt.subplots(figsize = (10,10))
cv = KFold(n_splits=5, shuffle=True,random_state=42)
plot_cv_indices(cv, X_train, y_train, ax, n_splits)

savepdf(fig,'cv_results')

# Build the model based on your best hyperparameters

In [ ]:
#500 1 for VGG fine-tuning
scaler=StandardScaler()
cls = RandomForestRegressor(bootstrap=True, n_jobs=-1, random_state = 40,max_features ='auto',n_estimators =500,min_samples_leaf=1 )
our_pipeline=Pipeline([('scaler',scaler),('cls',cls)])
our_pipeline.fit(X_train,y_train.ravel())

# Visualization

In [ ]:
Rsquared = stats.spearmanr((our_pipeline.predict(X_train)),y_train.ravel())[0]
pvalue = stats.spearmanr((our_pipeline.predict(X_train)),y_train.ravel())[1]
Rsquared_pearson = stats.pearsonr((our_pipeline.predict(X_train)),y_train.ravel())[0]
pvalue_pearson = stats.pearsonr((our_pipeline.predict(X_train)),y_train.ravel())[1]

In [ ]:
plt.rcParams.update({'mathtext.default':  'regular' })
my_prediction = our_pipeline.predict(X_train)
RMSE = round(np.sqrt(metrics.mean_squared_error(y_train, my_prediction)),1)
MAE = round(metrics.mean_absolute_error(y_train, my_prediction),1)
fig, ax = plt.subplots(figsize = (10,10))
ax.set_yscale('log')
ax.set_xscale('log')
ax.scatter(y_train, my_prediction,color = 'orange', edgecolors=(0, 0, 0),  s = 100)
ax.plot([0, 400], [0, 400], 'k--', lw=4)
ax.set_xlabel('True $PM_{2.5}$ ($\mu $g m$^{-3}$)', size = 20)
ax.set_ylabel('Predicted $PM_{2.5}$ ($\mu $g m$^{-3}$)', size = 20)
ax.tick_params(labelsize = 20)
fig.text(0.15, 0.83, 'Spearman r = '+ str(round(Rsquared,2)) + ' (p-value = '+ str(round(pvalue,7))+')', color='black', weight='roman',
fontsize=20)

plt.axis('tight')
fig.text(0.15, 0.79, 'Pearson r = '+ str(round(Rsquared_pearson,2)) + ' (p-value = '+ str(round(pvalue_pearson,100)) + ')', color='black', weight='roman',
fontsize=20)

fig.text(0.15, 0.75, 'RMSE = '+ str(RMSE), color='black', weight='roman',
fontsize=20)
fig.text(0.15, 0.71, 'MAE = '+ str(MAE), color='black', weight='roman',
fontsize=20)
fig.text(0.15, 0.67, 'NRMSE = '+ str(round(RMSE/np.mean(y_train)*100,1))+'%', color='black', weight='roman',
fontsize=20)
fig.text(0.15, 0.63, 'NMAE = '+ str(round(MAE/np.mean(y_train)*100,1))+'%', color='black', weight='roman',
fontsize=20)
fig.text(0.567, 0.135, 'Beijing training dataset', bbox=dict(facecolor='grey', alpha=0.9),color='black', weight='roman',
fontsize=20)

#plt.gca().set_aspect('equal', adjustable='box')
plt.show()
#savepdf(fig,'Satellite_Beijing_VGGtune_training_best')

In [ ]:
Rsquared = stats.spearmanr((our_pipeline.predict(X_test)),y_test.ravel())[0]
pvalue = stats.spearmanr((our_pipeline.predict(X_test)),y_test.ravel())[1]
Rsquared_pearson = stats.pearsonr((our_pipeline.predict(X_test)),y_test.ravel())[0]
pvalue_pearson = stats.pearsonr((our_pipeline.predict(X_test)),y_test.ravel())[1]

In [ ]:
plt.rcParams.update({'mathtext.default':  'regular' })
my_prediction = our_pipeline.predict(X_test)
RMSE = round(np.sqrt(metrics.mean_squared_error(y_test, my_prediction)),1)
MAE = round(metrics.mean_absolute_error(y_test, my_prediction),1)
fig, ax = plt.subplots(figsize = (10,10))
ax.set_yscale('log')
ax.set_xscale('log')
ax.scatter(y_test, my_prediction, color = 'royalblue',edgecolors=(0, 0, 0), s = 100)
ax.plot([0, 400], [0, 400], 'k--', lw=4)
ax.set_xlabel('True $PM_{2.5}$ ($\mu $g m$^{-3}$)', size = 30)
ax.set_ylabel('Predicted $PM_{2.5}$ ($\mu $g m$^{-3}$)', size = 30)
ax.tick_params(labelsize = 30)
fig.text(0.15, 0.85, 'Spearman r = '+ str(round(Rsquared,2)) + ' (p-value = '+ str(round(pvalue,7))+')', color='black', weight='roman',
fontsize=25)

plt.axis('tight')
fig.text(0.15, 0.81, 'Pearson r = '+ str(round(Rsquared_pearson,2)) + ' (p-value = '+ str(round(pvalue_pearson,2)) + ')', color='black', weight='roman',
fontsize=25)

fig.text(0.15, 0.77, 'RMSE = '+ str(RMSE), color='black', weight='roman',
fontsize=25)
fig.text(0.15, 0.73, 'MAE = '+ str(MAE), color='black', weight='roman',
fontsize=25)
fig.text(0.15, 0.69, 'NRMSE = '+ str(round(RMSE/np.mean(y_test)*100,1))+'%', color='black', weight='roman',
fontsize=25)
fig.text(0.15, 0.65, 'NMAE = '+ str(round(MAE/np.mean(y_test)*100,1))+'%', color='black', weight='roman',
fontsize=25)
fig.text(0.49, 0.14, 'Beijing test dataset', bbox=dict(facecolor='grey', alpha=0.9),color='black', weight='roman',
fontsize=30)
#plt.gca().set_aspect('equal', adjustable='box')
plt.gcf().subplots_adjust(left=0.15)
plt.show()
savepdf(fig,'Satellite_Beijing_VGGtune_test_best')
#savepdf(fig,'Satellite_Beijing_VGG16')

# RF Importance

In [ ]:
importances = cls.feature_importances_

std = np.std([tree.feature_importances_ for tree in cls.estimators_],
             axis=0)
indices = np.argsort(importances)[::-1]

importances

#indices[:10]

importance_indices = []
for i in range(128):
    importance_indices.append('Image_'+str(i))
importance_indices.append('Temperature')
importance_indices.append('RH')
importance_indices.append('Wind speed')
importance_indices.append('SLP')

In [ ]:
fig=plt.figure(figsize = (20,20))

num_feature = 10
plt.bar(range(num_feature), importances[indices][:num_feature],
      color="darkred", align="center")
plt.xticks(range(num_feature), np.array(importance_indices)[indices][:num_feature], rotation=45,size=35)
plt.yticks(size=40, rotation=45)
plt.xlabel('Top'+ str(num_feature) +' features',size = 40)
plt.ylabel('Importance',size = 40)
fig.text(0.695, 0.862, 'Beijing test dataset', bbox=dict(facecolor='grey', alpha=0.9),color='black', weight='roman',
fontsize=30)
fig.subplots_adjust(bottom=0.3, left = 0.2)
plt.show()
#savepdf(fig,'Feature_importances VGG16tune_Beijing_top10')

# Out-of-bag sample (US EMBASSY) test

In [ ]:
my_X_images_folder = './X_image_embassy'
image_files = [f for f in listdir(my_X_images_folder) if isfile(join(my_X_images_folder, f)) and '.DS_Store' not in f ]
image_files_sorted = sorted(image_files, key = lambda x: int(x.split('.')[0].split('_')[2]))

image_files_sorted

In [ ]:
X_image_embassy = []
image_size = 224
for image_file in image_files_sorted:
    img = image.load_img(my_X_images_folder+'/'+image_file,target_size=(image_size , image_size ))
    x = image.img_to_array(img) #(224,224,3)
    x = np.expand_dims(x, axis=0) #(1,224,224,3)
    x = preprocess_input(x)
    X_image_embassy.append(x)

In [ ]:
y_PM25_embassy = []
with open("./y_PM25_embassy.txt", "r") as f:
    for line in f:
        y_PM25_embassy.append(np.float32(float(line.strip())))

In [ ]:
len(X_image_embassy)

In [ ]:
X_image_embassy_features = extract_model.predict(np.array(X_image_embassy).reshape(-1,image_size ,image_size ,3))

X_image_embassy_features

In [ ]:
meteo_feature_embassy = np.load('meteo_feature_embassy.npy')
features_meteo_embassy = np.append(X_image_embassy_features,meteo_feature_embassy,axis = 1)



mask = ~np.any(np.isnan(features_meteo_embassy), axis=1)
features_meteo_embassy = features_meteo_embassy[mask]

y_all_embassy= np.array(y_PM25_embassy).reshape([373,-1])[mask]

features_meteo_embassy

y_all_embassy

In [ ]:
prediction_embassy = our_pipeline.predict(features_meteo_embassy)

np.sqrt(round(metrics.mean_squared_error(y_all_embassy, prediction_embassy),0))

In [ ]:
Rsquared = stats.spearmanr(prediction_embassy,y_all_embassy.ravel())[0]
pvalue = stats.spearmanr(prediction_embassy,y_all_embassy.ravel())[1]
Rsquared_pearson = stats.pearsonr(prediction_embassy,y_all_embassy.ravel())[0]
pvalue_pearson = stats.pearsonr(prediction_embassy,y_all_embassy.ravel())[1]

In [ ]:
RMSE = round(np.sqrt(metrics.mean_squared_error(y_all_embassy, prediction_embassy)),1)
MAE = round(metrics.mean_absolute_error(y_all_embassy, prediction_embassy),1)
fig, ax = plt.subplots(figsize = (10,10))
ax.set_yscale('log')
ax.set_xscale('log')
ax.scatter(y_all_embassy, prediction_embassy, color = 'plum',edgecolors=(0, 0, 0), s = 100)
ax.plot([0, 400], [0, 400], 'k--', lw=4)
ax.set_xlabel('True $PM_{2.5}$ ($\mu $g m$^{-3}$)', size = 30)
ax.set_ylabel('Predicted $PM_{2.5}$ ($\mu $g m$^{-3}$)', size = 30)
ax.tick_params(labelsize = 30)
fig.text(0.15, 0.85, 'Spearman r = '+ str(round(Rsquared,2)) + ' (p-value = '+ str(round(pvalue,7))+')', color='black', weight='roman',
fontsize=25)

plt.axis('tight')
fig.text(0.15, 0.81, 'Pearson r = '+ str(round(Rsquared_pearson,2)) + ' (p-value = '+ str(round(pvalue_pearson,2)) + ')', color='black', weight='roman',
fontsize=25)

fig.text(0.15, 0.77, 'RMSE = '+ str(RMSE), color='black', weight='roman',
fontsize=25)
fig.text(0.15, 0.73, 'MAE = '+ str(MAE), color='black', weight='roman',
fontsize=25)
fig.text(0.15, 0.69, 'NRMSE = '+ str(round(RMSE/np.mean(y_all_embassy)*100,1))+'%', color='black', weight='roman',
fontsize=25)
fig.text(0.15, 0.65, 'NMAE = '+ str(round(MAE/np.mean(y_all_embassy)*100,1))+'%', color='black', weight='roman',
fontsize=25)
fig.text(0.305, 0.14, 'Embassy out-of-bag dataset', bbox=dict(facecolor='grey', alpha=0.9),color='black', weight='roman',
fontsize=30)
plt.gcf().subplots_adjust(left=0.15)
plt.show()
savepdf(fig,'Embassy_out_of_bag_VGG16fine')

# Finally Shanghai Data

In [ ]:
my_X_images_folder = './X_image_Shanghai'
image_files = [f for f in listdir(my_X_images_folder) if isfile(join(my_X_images_folder, f)) and '.DS_Store' not in f ]
image_files_sorted = sorted(image_files, key = lambda x: int(x.split('.')[0].split('_')[2]))

image_files_sorted

In [ ]:
X_image_shanghai = []
for image_file in image_files_sorted:
    img = image.load_img(my_X_images_folder+'/'+image_file,target_size=(image_size , image_size ))
    x = image.img_to_array(img) #(224,224,3)
    x = np.expand_dims(x, axis=0) #(1,224,224,3)
    x = preprocess_input(x)
    X_image_shanghai.append(x)

In [ ]:
y_PM25_shanghai = []
with open("./y_PM25_Shanghai.txt", "r") as f:
    for line in f:
        y_PM25_shanghai.append(np.float32(float(line.strip())))

In [ ]:
X_image_shanghai_features = extract_model.predict(np.array(X_image_shanghai).reshape(-1,image_size ,image_size ,3))

X_image_shanghai_features

In [ ]:
meteo_feature_shanghai = np.load('meteo_feature_shanghai.npy')
features_meteo_shanghai = np.append(X_image_shanghai_features,meteo_feature_shanghai,axis = 1)



mask = ~np.any(np.isnan(features_meteo_shanghai), axis=1)
features_meteo_shanghai = features_meteo_shanghai[mask]

y_all_shanghai= np.array(y_PM25_shanghai).reshape([1897,-1])[mask]

prediction_shanghai = our_pipeline.predict(features_meteo_shanghai)

In [ ]:
Rsquared = stats.spearmanr(prediction_shanghai ,y_all_shanghai .ravel())[0]
pvalue = stats.spearmanr(prediction_shanghai ,y_all_shanghai .ravel())[1]
Rsquared_pearson = stats.pearsonr(prediction_shanghai ,y_all_shanghai .ravel())[0]
pvalue_pearson = stats.pearsonr(prediction_shanghai ,y_all_shanghai .ravel())[1]

In [ ]:
RMSE = round(np.sqrt(metrics.mean_squared_error(y_all_shanghai , prediction_shanghai)),1)
MAE = round(metrics.mean_absolute_error(y_all_shanghai , prediction_shanghai),1)
fig, ax = plt.subplots(figsize = (10,10))
ax.set_yscale('log')
ax.set_xscale('log')
ax.scatter(y_all_shanghai , prediction_shanghai, color = 'gold',edgecolors=(0, 0, 0), s = 100)
ax.plot([0, 400], [0, 400], 'k--', lw=4)
ax.set_xlabel('True $PM_{2.5}$ ($\mu $g m$^{-3}$)', size = 30)
ax.set_ylabel('Predicted $PM_{2.5}$ ($\mu $g m$^{-3}$)', size = 30)
ax.tick_params(labelsize = 30)
fig.text(0.15, 0.85, 'Spearman r = '+ str(round(Rsquared,2)) + ' (p-value = '+ str(round(pvalue,7))+')', color='black', weight='roman',
fontsize=25)

plt.axis('tight')
fig.text(0.15, 0.81, 'Pearson r = '+ str(round(Rsquared_pearson,2)) + ' (p-value = '+ str(round(pvalue_pearson,2)) + ')', color='black', weight='roman',
fontsize=25)

fig.text(0.15, 0.77, 'RMSE = '+ str(RMSE), color='black', weight='roman',
fontsize=25)
fig.text(0.15, 0.73, 'MAE = '+ str(MAE), color='black', weight='roman',
fontsize=25)
fig.text(0.15, 0.69, 'NRMSE = '+ str(round(RMSE/np.mean(y_all_shanghai)*100,1))+'%', color='black', weight='roman',
fontsize=25)
fig.text(0.15, 0.65, 'NMAE = '+ str(round(MAE/np.mean(y_all_shanghai)*100,1))+'%', color='black', weight='roman',
fontsize=25)
fig.text(0.258, 0.14, 'Shanghai dataset (no training)', bbox=dict(facecolor='grey', alpha=0.9),color='black', weight='roman',
fontsize=30)
plt.gcf().subplots_adjust(left=0.15)
plt.show()
savepdf(fig,'Shanghai_VGG16fine_no_training')

# For shanghai, only re-train random forest then

In [ ]:
meteo_feature_shanghai = np.load('meteo_feature_shanghai.npy')
features_meteo_shanghai = np.append(X_image_shanghai_features,meteo_feature_shanghai,axis = 1)



mask = ~np.any(np.isnan(features_meteo_shanghai), axis=1)
features_meteo_shanghai = features_meteo_shanghai[mask]

y_all_shanghai= np.array(y_PM25_shanghai).reshape([1897,-1])[mask]

In [ ]:
## load in site labels and time index to see the seasonal trend and site-specific trend
time_stamps = np.load('time_stamp_shanghai.npy')
site_labels = np.load('site_label_shanghai.npy')

time_stamps = np.array([float(x) for x in time_stamps ])
site_labels = np.array([float(x) for x in site_labels ])

features_meteo = np.append(features_meteo_shanghai,time_stamps.reshape(1897,1),axis = 1)
features_meteo = np.append(features_meteo_shanghai,site_labels.reshape(1897,1),axis = 1)

X_train_full, X_test_full, y_train, y_test = train_test_split(features_meteo , y_all_shanghai, test_size=0.2, random_state=0)
X_train = X_train_full[:, :-2]
X_test = X_test_full[:, :-2]

In [ ]:
scaler=StandardScaler()
cls = RandomForestRegressor(bootstrap=True, n_jobs=-1, random_state = 40,max_features ='auto',n_estimators =500,min_samples_leaf=1 )
our_pipeline=Pipeline([('scaler',scaler),('cls',cls)])
our_pipeline.fit(X_train,y_train.ravel())

In [ ]:
Rsquared = stats.spearmanr((our_pipeline.predict(X_test)),y_test.ravel())[0]
pvalue = stats.spearmanr((our_pipeline.predict(X_test)),y_test.ravel())[1]
Rsquared_pearson = stats.pearsonr((our_pipeline.predict(X_test)),y_test.ravel())[0]
pvalue_pearson = stats.pearsonr((our_pipeline.predict(X_test)),y_test.ravel())[1]

In [ ]:
my_prediction = our_pipeline.predict(X_test)
RMSE = round(np.sqrt(metrics.mean_squared_error(y_test, my_prediction)),1)
MAE = round(metrics.mean_absolute_error(y_test, my_prediction),1)
fig, ax = plt.subplots(figsize = (10,10))
ax.set_yscale('log')
ax.set_xscale('log')
ax.scatter(y_test, my_prediction, color = 'coral',edgecolors=(0, 0, 0), s = 100)
ax.plot([0, 200], [0, 200], 'k--', lw=4)
ax.set_xlabel('True $PM_{2.5}$ ($\mu $g m$^{-3}$)', size = 30)
ax.set_ylabel('Predicted $PM_{2.5}$ ($\mu $g m$^{-3}$)', size = 30)
ax.tick_params(labelsize = 30)
fig.text(0.15, 0.85, 'Spearman r = '+ str(round(Rsquared,2)) + ' (p-value = '+ str(round(pvalue,7))+')', color='black', weight='roman',
fontsize=25)

plt.axis('tight')
fig.text(0.15, 0.81, 'Pearson r = '+ str(round(Rsquared_pearson,2)) + ' (p-value = '+ str(round(pvalue_pearson,2)) + ')', color='black', weight='roman',
fontsize=25)

fig.text(0.15, 0.77, 'RMSE = '+ str(RMSE), color='black', weight='roman',
fontsize=25)
fig.text(0.15, 0.73, 'MAE = '+ str(MAE), color='black', weight='roman',
fontsize=25)
fig.text(0.15, 0.69, 'NRMSE = '+ str(round(RMSE/np.mean(y_test)*100,1))+'%', color='black', weight='roman',
fontsize=25)
fig.text(0.15, 0.65, 'NMAE = '+ str(round(MAE/np.mean(y_test)*100,1))+'%', color='black', weight='roman',
fontsize=25)
fig.text(0.21, 0.137, 'Shanghai dataset (RF re-trained)', bbox=dict(facecolor='grey', alpha=0.9),color='black', weight='roman',
fontsize=30)
plt.gcf().subplots_adjust(left=0.15)
#plt.gca().set_aspect('equal', adjustable='box')
plt.show()
savepdf(fig,'Shanghai_VGGtune_RFretrain_best')